**Realizando a conexão com o MongoDB**

In [1]:
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv


load_dotenv()


uri = os.getenv("MONGO_ATLAS_URL")

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


## Lendo os dados no MongoDB

In [2]:
db = client["db_produtos"]

In [3]:
collection = db["produtos"]

In [5]:
collection.find_one()

{'_id': ObjectId('678ec72da60f68a3f0d7705b'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'lat': -13.29,
 'lon': -41.71}

**Alterando os nomes dos campos de latitude e longitude**

In [6]:
collection.update_many({}, {'$rename': {"lat": "Latitude", "lon": "Longitude"}})

UpdateResult({'n': 9435, 'electionId': ObjectId('7fffffff0000000000000015'), 'opTime': {'ts': Timestamp(1737470455, 5018), 't': 21}, 'nModified': 9435, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1737470455, 5018), 'signature': {'hash': b'\xd0\xf4\xc6B\xdfm\x1eK\xab\xf6I\x9f^\xa0\xf8\x14\xaf\x15\xb0\x0e', 'keyId': 7401159263046860802}}, 'operationTime': Timestamp(1737470455, 5018), 'updatedExisting': True}, acknowledged=True)

In [7]:
collection.find_one()

{'_id': ObjectId('678ec72da60f68a3f0d7705b'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

## Aplicando transformações: filtrando registros da categoria livros

Agora, vamos executar uma consulta para obter apenas os dados relacionados à categoria de **livros**.

In [8]:
collection.distinct("Categoria do Produto")

['brinquedos',
 'eletrodomesticos',
 'eletronicos',
 'esporte e lazer',
 'instrumentos musicais',
 'livros',
 'moveis',
 'utilidades domesticas']

In [12]:
query = {"Categoria do Produto": "livros"}

In [16]:
book_list = []
for doc in collection.find(query):
    book_list.append(doc)

**Salvando os dados em um DataFrame**

In [17]:
import pandas as pd


df = pd.DataFrame(book_list)
df.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,678ec72da60f68a3f0d7705b,Modelagem preditiva,livros,92.45,5.609697,01/01/2020,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,678ec72da60f68a3f0d7705c,Iniciando em programação,livros,43.84,0.000000,01/01/2020,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,678ec72da60f68a3f0d7706e,Iniciando em programação,livros,63.25,3.894137,01/01/2022,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,678ec72da60f68a3f0d7707c,Ciência de dados com python,livros,86.13,5.273176,01/02/2021,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,678ec72da60f68a3f0d7707e,Ciência de dados com python,livros,72.75,1.458158,01/02/2021,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


## Formatando as datas

Para adicionar dados do tipo DATE em um banco de dados MySQL, é necessário que a data esteja no formato **%Y-%m-%d (ano-mês-dia)**. Portanto, realizaremos uma transformação na coluna "Data de Compra" para ajustá-la a esse formato. Dessa forma, poderemos armazenar corretamente as datas no banco de dados MySQL, garantindo consistência e facilitando consultas baseadas em datas posteriormente.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     742 non-null    object 
 1   Produto                 742 non-null    object 
 2   Categoria do Produto    742 non-null    object 
 3   Preço                   742 non-null    float64
 4   Frete                   742 non-null    float64
 5   Data da Compra          742 non-null    object 
 6   Vendedor                742 non-null    object 
 7   Local da compra         742 non-null    object 
 8   Avaliação da compra     742 non-null    int64  
 9   Tipo de pagamento       742 non-null    object 
 10  Quantidade de parcelas  742 non-null    int64  
 11  Latitude                742 non-null    float64
 12  Longitude               742 non-null    float64
dtypes: float64(4), int64(2), object(7)
memory usage: 75.5+ KB


In [20]:
df['Data da Compra'] = pd.to_datetime(df['Data da Compra'], format="%d/%m/%Y")

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _id                     742 non-null    object        
 1   Produto                 742 non-null    object        
 2   Categoria do Produto    742 non-null    object        
 3   Preço                   742 non-null    float64       
 4   Frete                   742 non-null    float64       
 5   Data da Compra          742 non-null    datetime64[ns]
 6   Vendedor                742 non-null    object        
 7   Local da compra         742 non-null    object        
 8   Avaliação da compra     742 non-null    int64         
 9   Tipo de pagamento       742 non-null    object        
 10  Quantidade de parcelas  742 non-null    int64         
 11  Latitude                742 non-null    float64       
 12  Longitude               742 non-null    float64   

In [24]:
df['Data da Compra'] = df['Data da Compra'].dt.strftime("%Y-%m-%d")

In [25]:
df.to_csv("../data/books_data.csv", index=False)

## Aplicando transformações: filtrando produtos vendidos a partir de 2021

Nesta etapa, faremos uma query para selecionar apenas os produtos vendidos a partir do ano de 2021. Utilizaremos um filtro na coluna de data de compra para obter os registros que atendam a essa condição.

In [26]:
collection.find_one()

{'_id': ObjectId('678ec72da60f68a3f0d7705b'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

In [27]:
query = {"Data da Compra": {'$regex': '/202[1-9]'}}

In [29]:
data_from_2021_onwards = []
for doc in collection.find(query):
    data_from_2021_onwards.append(doc)

**Formatando as datas**

In [30]:
df_products_from_2021_onwards = pd.DataFrame(data_from_2021_onwards)
df_products_from_2021_onwards.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,678ec72da60f68a3f0d77064,Xadrez de madeira,brinquedos,25.23,0.000000,01/01/2021,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,678ec72da60f68a3f0d77065,Impressora,eletronicos,322.04,14.732100,01/01/2021,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,678ec72da60f68a3f0d77066,Mesa de centro,moveis,282.22,12.611805,01/01/2021,João Souza,SP,3,boleto,1,-22.19,-48.79
3,678ec72da60f68a3f0d77067,Tablet ABXY,eletronicos,1100.42,68.451348,01/01/2021,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,678ec72da60f68a3f0d77068,Fogão,eletrodomesticos,791.81,42.444626,01/01/2021,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [31]:
df_products_from_2021_onwards['Data da Compra'] = pd.to_datetime(df_products_from_2021_onwards['Data da Compra'], format="%d/%m/%Y")
df_products_from_2021_onwards['Data da Compra'] = df_products_from_2021_onwards['Data da Compra'].dt.strftime("%Y-%m-%d")
df_products_from_2021_onwards.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,678ec72da60f68a3f0d77064,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,678ec72da60f68a3f0d77065,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,678ec72da60f68a3f0d77066,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,678ec72da60f68a3f0d77067,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,678ec72da60f68a3f0d77068,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [32]:
df_products_from_2021_onwards.to_csv('../data/products_from_2021_onwards.csv', index=False)